In [1]:
#! /usr/bin/env python3

import rospy
import time
from sensor_msgs.msg import LaserScan
from visualization_msgs.msg import MarkerArray
from nav_msgs.msg import Odometry
from tf.transformations import euler_from_quaternion, quaternion_from_euler

from topometricmap.topo_map import *
from topometricmap.utils import *
from floor_plan.floor_plan import *
from floor_plan.floorplan_extraction import *
from floor_plan.utils import *

EMBEDDING_DIM = 512

scan = None
def scan_cb(msg: LaserScan):
    global scan
    scan = msg

euler = None
movement = 0.0
def odom_cb(msg: Odometry):
    global euler, movement
    q = (msg.pose.pose.orientation.x, msg.pose.pose.orientation.y, msg.pose.pose.orientation.z, msg.pose.pose.orientation.w)
    movement += np.linalg.norm(np.asarray( (msg.twist.twist.linear.x, msg.twist.twist.linear.y), dtype=float))
    euler = euler_from_quaternion(q, axes='sxyz')

rospy.init_node("floor_plan", anonymous=True)


In [6]:

floor_plan = FloorPlan("/home/rtu/catkin_clip_topo/src/floor_plane/topdown_floors3.png")
floor_plan.InitBelief(0)
floor_plan.InitSimilarity()
# floor_plan.PropogatePosition()

Takes 831.0558795928955 to parse the floor plan...
Process vertices: **********100%Takes 6454.367637634277 for laserscan sampling...

Process edges: **********100%

In [2]:
marker_pub = rospy.Publisher("floor_plan", MarkerArray, queue_size=10)
rospy.Subscriber("scan_aug", LaserScan, callback=scan_cb, queue_size=1)
rospy.Subscriber("odom", Odometry, callback=odom_cb, queue_size=1)

rospy.Rate(1).sleep()

Takes 794.7967052459717 to parse the floor plan...
Process vertices: **********100%Takes 5579.517126083374 for laserscan sampling...

Process edges: **********100%

In [8]:
markers = floor_plan.Visuzlize(show_laser=True)
marker_pub.publish(markers)

In [7]:
t1 = time.time()
if scan is not None and movement>2.0:

    # align the
    scan.angle_min += euler[2]
    scan.angle_max += euler[2]
    floor_plan.UpdateSimilarity(scan)
    floor_plan.UpdateBelief(move=0.35)
    movement=0.0
t2 = time.time()

markers = floor_plan.Visuzlize()
marker_pub.publish(markers)
t3 = time.time()

print(f"Update time: {(t2-t1)*1000.0}")
print(f"Visualize time: {(t3-t2)*1000.0}")

Update time: 0.12254714965820312
Visualize time: 14.792680740356445


In [22]:
floor_plan.PropogatePosition(max_distance=-1)

In [23]:
markers = floor_plan.Visuzlize(show_laser=True)
marker_pub.publish(markers)

In [15]:
floor_plan.nodes[27].pose_opt

array([ 0.8, -3.4,  0. ,  0. ,  0. ,  0. ])

In [16]:
floor_plan.nodes[27].pose_odom

array([ 2.6, -5.5,  0. ,  0. ,  0. ,  0. ])

In [17]:
floor_plan.nodes[29].pose_opt

array([ 3.8, -5.8,  0. ,  0. ,  0. ,  0. ])

In [18]:
floor_plan.nodes[29].pose_odom

array([ 3.8, -5.8,  0. ,  0. ,  0. ,  0. ])

In [13]:
floor_plan.nodes[29].RelativePose2D(floor_plan.nodes[27])

array([ 1.2, -0.3,  0. ,  0. ,  0. ,  0. ])